In [ ]:
import cohere
import os
import numpy
import math

from dotenv import load_dotenv


load_dotenv()
key = os.getenv("openai_api_key")

co = cohere.Client(key)




In [ ]:
file = open("C:\\Users\\bouta\\OneDrive\\Bureau\\hackathons\\Think AI\\src\\util\\data.txt", "r")

texts = []
for line in file:
    
    texts.append("recipe title: "+line.split(":")[0]+"  ingridients: "+line.split(":")[1])
    
print(texts)    

In [31]:
response = co.embed(
    texts=texts,
    model='small',
)
embeddings=response.embeddings
for embedding in embeddings:
    print(embedding)




50


In [ ]:
""" let's test here first """
response = co.embed(
    texts=["recipe title: fish Tagine with Preserved Lemons and Olives  ingridients:  fish thighs, onions, garlic, preserved lemons, green olives, ginger, cumin, coriander, parsley, olive oil, salt, black pepper"],
    model='small',
)
embeddingForRecipeTest=response.embeddings


